In [1]:
!pip install autogluon

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import datetime

# DATA_PATH = "/content/drive/MyDrive/DACON/제주도/data/"
DATA_PATH = "/content/drive/MyDrive/DACON_JEJU/data/"
SEED = 42

In [4]:
train_df = pd.read_csv(f"{DATA_PATH}train.csv")
trade_df = pd.read_csv(f"{DATA_PATH}international_trade.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           59397 non-null  object 
 1   timestamp    59397 non-null  object 
 2   item         59397 non-null  object 
 3   corporation  59397 non-null  object 
 4   location     59397 non-null  object 
 5   supply(kg)   59397 non-null  float64
 6   price(원/kg)  59397 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.2+ MB


In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092 entries, 0 to 1091
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           1092 non-null   object
 1   timestamp    1092 non-null   object
 2   item         1092 non-null   object
 3   corporation  1092 non-null   object
 4   location     1092 non-null   object
dtypes: object(5)
memory usage: 42.8+ KB


In [7]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['year'] = train_df['timestamp'].dt.year
train_df['month'] = train_df['timestamp'].dt.month
train_df['week'] = train_df['timestamp'].dt.isocalendar().loc[:, 'week']
train_df['day'] = train_df['timestamp'].dt.day
train_df['day_of_week'] = train_df['timestamp'].dt.day_of_week
train_df['day_of_year'] = train_df['timestamp'].dt.day_of_year
train_df['sin_dayofweek'] = np.sin(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['cos_dayofweek'] = np.cos(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,0.974928,-0.222521
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,2,0.433884,-0.900969
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,3,-0.433884,-0.900969
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,4,-0.974928,-0.222521
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,5,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,27,0,58,0.781831,0.623490
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,28,1,59,0.974928,-0.222521
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,1,2,60,0.433884,-0.900969
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,2,3,61,-0.433884,-0.900969


In [8]:
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df['year'] = test_df['timestamp'].dt.year
test_df['month'] = test_df['timestamp'].dt.month
test_df['week'] = test_df['timestamp'].dt.isocalendar().loc[:, 'week']
test_df['day'] = test_df['timestamp'].dt.day
test_df['day_of_week'] = test_df['timestamp'].dt.day_of_week
test_df['day_of_year'] = test_df['timestamp'].dt.day_of_year
test_df['sin_dayofweek'] = np.sin(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['cos_dayofweek'] = np.cos(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,63,-7.818315e-01,0.623490
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,64,-2.449294e-16,1.000000
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,65,7.818315e-01,0.623490
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,66,9.749279e-01,-0.222521
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,67,4.338837e-01,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,13,27,0,86,7.818315e-01,0.623490
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,13,28,1,87,9.749279e-01,-0.222521
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,13,29,2,88,4.338837e-01,-0.900969
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,13,30,3,89,-4.338837e-01,-0.900969


In [9]:
train_df.rename(columns={'supply(kg)' : 'supply', 'price(원/kg)' : 'price'}, inplace=True)
train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,0.974928,-0.222521
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,2,0.433884,-0.900969
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,3,-0.433884,-0.900969
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,4,-0.974928,-0.222521
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,5,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,27,0,58,0.781831,0.623490
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,28,1,59,0.974928,-0.222521
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,1,2,60,0.433884,-0.900969
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,2,3,61,-0.433884,-0.900969


In [10]:
month_week_mean = train_df.groupby(['item', 'corporation', 'location', 'month', 'week'])['price'].mean().reset_index()
month_week_mean = month_week_mean.rename(columns={'price' : 'month_week_mean'})
train_df = train_df.merge(month_week_mean, on=['item','corporation', 'location', 'month', 'week'], how='left')
test_df = test_df.merge(month_week_mean, on=['item','corporation', 'location', 'month' , 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,0.974928,-0.222521,1106.90625
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,2,0.433884,-0.900969,1106.90625
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,3,-0.433884,-0.900969,1106.90625
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,4,-0.974928,-0.222521,1106.90625
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,5,-0.781831,0.623490,1106.90625


In [11]:
month_week_max = train_df.groupby(['item', 'corporation', 'location', 'month', 'week'])['price'].max().reset_index()
month_week_max = month_week_max.rename(columns={'price' : 'month_week_max'})
train_df = train_df.merge(month_week_max, on=['item','corporation', 'location', 'month', 'week'], how='left')
test_df = test_df.merge(month_week_max, on=['item','corporation', 'location', 'month' , 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,0.974928,-0.222521,1106.90625,3569.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,2,0.433884,-0.900969,1106.90625,3569.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,3,-0.433884,-0.900969,1106.90625,3569.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,4,-0.974928,-0.222521,1106.90625,3569.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,5,-0.781831,0.623490,1106.90625,3569.0


In [12]:
month_week_std = train_df.groupby(['item', 'corporation', 'location', 'month', 'week'])['price'].std().reset_index()
month_week_std = month_week_std.rename(columns={'price' : 'month_week_std'})
train_df = train_df.merge(month_week_std, on=['item','corporation', 'location', 'month', 'week'], how='left')
test_df = test_df.merge(month_week_std, on=['item','corporation', 'location', 'month' , 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,0.974928,-0.222521,1106.90625,3569.0,878.46238
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,2,0.433884,-0.900969,1106.90625,3569.0,878.46238
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,3,-0.433884,-0.900969,1106.90625,3569.0,878.46238
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,4,-0.974928,-0.222521,1106.90625,3569.0,878.46238
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,5,-0.781831,0.623490,1106.90625,3569.0,878.46238


In [13]:
month_mean = train_df.groupby(['item', 'corporation', 'location', 'month'])['price'].mean().reset_index()
month_mean = month_mean.rename(columns={'price' : 'month_mean'})
train_df = train_df.merge(month_mean, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_mean, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,2,0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,3,-0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,4,-0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,5,-0.781831,0.623490,1106.90625,3569.0,878.46238,1403.858065


In [14]:
month_max = train_df.groupby(['item', 'corporation', 'location', 'month'])['price'].max().reset_index()
month_max = month_max.rename(columns={'price' : 'month_max'})
train_df = train_df.merge(month_max, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_max, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean,month_max
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,1,0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,2,0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,3,-0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,4,-0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,5,-0.781831,0.623490,1106.90625,3569.0,878.46238,1403.858065,5685.0


In [15]:
month_std = train_df.groupby(['item', 'corporation', 'location', 'month'])['price'].std().reset_index()
month_std = month_std.rename(columns={'price' : 'month_std'})
train_df = train_df.merge(month_std, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_std, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean,month_max,month_std
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1,1,0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,2,2,0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,3,3,-0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,4,4,-0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,5,5,-0.781831,0.623490,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903


In [16]:
year_month_mean = train_df.groupby(['item', 'corporation', 'location', 'month', 'year'])['price'].mean().reset_index()
year_month_mean

,item,corporation,location,month,year,price
0,BC,A,J,1,2019,1673.451613
1,BC,A,J,1,2020,1615.870968
2,BC,A,J,1,2021,2108.838710
3,BC,A,J,1,2022,3162.451613
4,BC,A,J,1,2023,1805.258065
...,...,...,...,...,...,...
1984,TG,E,S,11,2022,1094.966667
1985,TG,E,S,12,2019,1096.451613
1986,TG,E,S,12,2020,1160.645161
1987,TG,E,S,12,2021,1398.548387


In [17]:
year_month_mean['month'] = year_month_mean['month'] + 1
year_month_mean

,item,corporation,location,month,year,price
0,BC,A,J,2,2019,1673.451613
1,BC,A,J,2,2020,1615.870968
2,BC,A,J,2,2021,2108.838710
3,BC,A,J,2,2022,3162.451613
4,BC,A,J,2,2023,1805.258065
...,...,...,...,...,...,...
1984,TG,E,S,12,2022,1094.966667
1985,TG,E,S,13,2019,1096.451613
1986,TG,E,S,13,2020,1160.645161
1987,TG,E,S,13,2021,1398.548387


In [18]:
mask = year_month_mean['month'] == 13
year_month_mean.loc[mask, 'month'] = 1
year_month_mean.loc[mask, 'year'] = year_month_mean.loc[mask, 'year'] + 1
year_month_mean

,item,corporation,location,month,year,price
0,BC,A,J,2,2019,1673.451613
1,BC,A,J,2,2020,1615.870968
2,BC,A,J,2,2021,2108.838710
3,BC,A,J,2,2022,3162.451613
4,BC,A,J,2,2023,1805.258065
...,...,...,...,...,...,...
1984,TG,E,S,12,2022,1094.966667
1985,TG,E,S,1,2020,1096.451613
1986,TG,E,S,1,2021,1160.645161
1987,TG,E,S,1,2022,1398.548387


In [19]:
year_month_mean = year_month_mean.rename(columns={'price' : 'last_month_mean'})
train_df = train_df.merge(year_month_mean, on=['item','corporation', 'location', 'month', 'year'], how='left')
test_df = test_df.merge(year_month_mean, on=['item','corporation', 'location', 'month' , 'year'], how='left')

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean,month_max,month_std,last_month_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1,0.974928,-0.222521,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,2,0.433884,-0.900969,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,NaN
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,3,-0.433884,-0.900969,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,NaN
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,4,-0.974928,-0.222521,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,NaN
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,5,-0.781831,0.623490,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,...,58,0.781831,0.623490,412.384615,531.0,53.849386,354.177305,689.0,192.959888,364.290323
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,...,59,0.974928,-0.222521,412.384615,531.0,53.849386,354.177305,689.0,192.959888,364.290323
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,...,60,0.433884,-0.900969,348.500000,574.0,192.262293,332.102362,574.0,158.666708,434.857143
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,...,61,-0.433884,-0.900969,348.500000,574.0,192.262293,332.102362,574.0,158.666708,434.857143


In [20]:
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean,month_max,month_std,last_month_mean
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,63,-7.818315e-01,0.623490,2487.800000,4538.0,1676.428208,3215.417323,10044.0,2047.942430,1955.285714
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,64,-2.449294e-16,1.000000,2487.800000,4538.0,1676.428208,3215.417323,10044.0,2047.942430,1955.285714
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,65,7.818315e-01,0.623490,2563.142857,4838.0,1579.706037,3215.417323,10044.0,2047.942430,1955.285714
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,66,9.749279e-01,-0.222521,2563.142857,4838.0,1579.706037,3215.417323,10044.0,2047.942430,1955.285714
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,67,4.338837e-01,-0.900969,2563.142857,4838.0,1579.706037,3215.417323,10044.0,2047.942430,1955.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,13,27,0,86,7.818315e-01,0.623490,354.761905,552.0,143.423117,332.102362,574.0,158.666708,434.857143
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,13,28,1,87,9.749279e-01,-0.222521,354.761905,552.0,143.423117,332.102362,574.0,158.666708,434.857143
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,13,29,2,88,4.338837e-01,-0.900969,354.761905,552.0,143.423117,332.102362,574.0,158.666708,434.857143
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,13,30,3,89,-4.338837e-01,-0.900969,354.761905,552.0,143.423117,332.102362,574.0,158.666708,434.857143


In [21]:
month_mean = train_df.groupby(['item', 'corporation', 'location', 'month'])['price'].mean().reset_index()
month_1_mean = month_mean.loc[month_mean['month'] == 12]
month_1_mean

,item,corporation,location,month,price
11,BC,A,J,12,2292.362903
23,BC,A,S,12,2410.120968
35,BC,B,J,12,2014.129032
47,BC,B,S,12,79.637097
59,BC,C,J,12,2166.862903
71,BC,C,S,12,17.137097
83,BC,D,J,12,2546.209677
95,BC,E,J,12,2670.758065
107,BC,E,S,12,2748.701613
119,CB,A,J,12,421.298387


In [22]:
for item in train_df['item'].unique():
    for corporation in train_df['corporation'].unique():
        for location in train_df['location'].unique():
            try:
                mask_1 = (train_df['item'] == item) & (train_df['corporation'] == corporation) & (train_df['location'] == location)
                mask_2 = (month_1_mean['item'] == item) & (month_1_mean['corporation'] == corporation) & (month_1_mean['location'] == location)
                train_df[mask_1] = train_df[mask_1].fillna(month_1_mean.loc[mask_2, 'price'].values[0])
            except:
                continue

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,day_of_year,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean,month_max,month_std,last_month_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1,0.974928,-0.222521,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,968.604839
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,2,0.433884,-0.900969,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,968.604839
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,3,-0.433884,-0.900969,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,968.604839
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,4,-0.974928,-0.222521,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,968.604839
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,5,-0.781831,0.623490,1106.906250,3569.0,878.462380,1403.858065,5685.0,988.542903,968.604839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,...,58,0.781831,0.623490,412.384615,531.0,53.849386,354.177305,689.0,192.959888,364.290323
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,...,59,0.974928,-0.222521,412.384615,531.0,53.849386,354.177305,689.0,192.959888,364.290323
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,...,60,0.433884,-0.900969,348.500000,574.0,192.262293,332.102362,574.0,158.666708,434.857143
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,...,61,-0.433884,-0.900969,348.500000,574.0,192.262293,332.102362,574.0,158.666708,434.857143


In [23]:
month_mean_supply = train_df.groupby(['item', 'corporation', 'location', 'month'])['supply'].mean().reset_index()
month_mean_supply = month_mean_supply.rename(columns={'supply' : 'month_mean_supply'})
train_df = train_df.merge(month_mean_supply, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_mean_supply, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,sin_dayofweek,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean,month_max,month_std,last_month_mean,month_mean_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,-0.433884,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,-0.974928,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,-0.781831,0.623490,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839


In [24]:
month_std_supply = train_df.groupby(['item', 'corporation', 'location', 'month'])['supply'].std().reset_index()
month_std_supply = month_std_supply.rename(columns={'supply' : 'month_std_supply'})
train_df = train_df.merge(month_std_supply, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_std_supply, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,cos_dayofweek,month_week_mean,month_week_max,month_week_std,month_mean,month_max,month_std,last_month_mean,month_mean_supply,month_std_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,-0.900969,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,-0.222521,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.623490,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794


In [25]:
month_week_mean_supply = train_df.groupby(['item', 'corporation', 'location', 'month', 'week'])['supply'].mean().reset_index()
month_week_mean_supply = month_week_mean_supply.rename(columns={'supply' : 'month_week_mean_supply'})
train_df = train_df.merge(month_week_mean_supply, on=['item','corporation', 'location', 'month', 'week'], how='left')
test_df = test_df.merge(month_week_mean_supply, on=['item','corporation', 'location', 'month' , 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_week_mean,month_week_max,month_week_std,month_mean,month_max,month_std,last_month_mean,month_mean_supply,month_std_supply,month_week_mean_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,1106.90625,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3


In [26]:
month_week_std_supply = train_df.groupby(['item', 'corporation', 'location', 'month', 'week'])['supply'].std().reset_index()
month_week_std_supply = month_week_std_supply.rename(columns={'supply' : 'month_week_std_supply'})
train_df = train_df.merge(month_week_std_supply, on=['item','corporation', 'location', 'month', 'week'], how='left')
test_df = test_df.merge(month_week_std_supply, on=['item','corporation', 'location', 'month' , 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_week_max,month_week_std,month_mean,month_max,month_std,last_month_mean,month_mean_supply,month_std_supply,month_week_mean_supply,month_week_std_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,3569.0,878.46238,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661


In [27]:
train_df['total'] = train_df['supply'] * train_df['price']

month_week_mean_total = train_df.groupby(['item', 'corporation', 'location', 'month', 'week'])['total'].mean().reset_index()
month_week_mean_total = month_week_mean_total.rename(columns={'total' : 'month_week_mean_total'})
train_df = train_df.merge(month_week_mean_total, on=['item','corporation', 'location', 'month', 'week'], how='left')
test_df = test_df.merge(month_week_mean_total, on=['item','corporation', 'location', 'month' , 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_mean,month_max,month_std,last_month_mean,month_mean_supply,month_std_supply,month_week_mean_supply,month_week_std_supply,total,month_week_mean_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,104718528.0,5.244109e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,35200000.0,5.244109e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,1403.858065,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,40440000.0,5.244109e+07


In [28]:
month_week_std_total = train_df.groupby(['item', 'corporation', 'location', 'month', 'week'])['total'].std().reset_index()
month_week_std_total = month_week_std_total.rename(columns={'total' : 'month_week_std_total'})
train_df = train_df.merge(month_week_std_total, on=['item','corporation', 'location', 'month', 'week'], how='left')
test_df = test_df.merge(month_week_std_total, on=['item','corporation', 'location', 'month' , 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_max,month_std,last_month_mean,month_mean_supply,month_std_supply,month_week_mean_supply,month_week_std_supply,total,month_week_mean_total,month_week_std_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07,5.790161e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07,5.790161e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,104718528.0,5.244109e+07,5.790161e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,35200000.0,5.244109e+07,5.790161e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,5685.0,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,40440000.0,5.244109e+07,5.790161e+07


In [29]:
month_mean_total = train_df.groupby(['item', 'corporation', 'location', 'month'])['total'].mean().reset_index()
month_mean_total = month_mean_total.rename(columns={'total' : 'month_mean_total'})
train_df = train_df.merge(month_mean_total, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_mean_total, on=['item','corporation', 'location', 'month' ], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_std,last_month_mean,month_mean_supply,month_std_supply,month_week_mean_supply,month_week_std_supply,total,month_week_mean_total,month_week_std_total,month_mean_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07,5.790161e+07,4.952021e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07,5.790161e+07,4.952021e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,104718528.0,5.244109e+07,5.790161e+07,4.952021e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,35200000.0,5.244109e+07,5.790161e+07,4.952021e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,988.542903,968.604839,29435.214839,31616.201794,35149.3,41096.586661,40440000.0,5.244109e+07,5.790161e+07,4.952021e+07


In [30]:
month_std_total = train_df.groupby(['item', 'corporation', 'location', 'month'])['total'].std().reset_index()
month_std_total = month_std_total.rename(columns={'total' : 'month_std_total'})
train_df = train_df.merge(month_std_total, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_std_total, on=['item','corporation', 'location', 'month' ], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,last_month_mean,month_mean_supply,month_std_supply,month_week_mean_supply,month_week_std_supply,total,month_week_mean_total,month_week_std_total,month_mean_total,month_std_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,968.604839,29435.214839,31616.201794,35149.3,41096.586661,0.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,968.604839,29435.214839,31616.201794,35149.3,41096.586661,104718528.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,968.604839,29435.214839,31616.201794,35149.3,41096.586661,35200000.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,968.604839,29435.214839,31616.201794,35149.3,41096.586661,40440000.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07


In [31]:
trade_df = trade_df[trade_df.품목명.str.contains('감귤|브로콜리|무|당근|양배추')]
trade_df = trade_df[(trade_df.품목명 != '무화과') & (trade_df.품목명 != '방울다다기 양배추')].reset_index(drop=True)
trade_df

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
1,2019-01,양배추,184650,94,395802,90,4
2,2019-01,당근,23150,22,7466150,2955,-2934
3,2019-01,감귤,58368,172,0,0,172
4,2019-02,꽃양배추와 브로콜리(broccoli),780,1,396870,399,-398
...,...,...,...,...,...,...,...
201,2023-02,꽃양배추와 브로콜리(broccoli),24,0,332640,352,-352
202,2023-02,양배추,13188,13,377456,104,-91
203,2023-02,당근,22510,20,9260020,3758,-3737
204,2023-02,순무,4000,4,2,0,4


In [32]:
trade_df['year'] = trade_df['기간'].apply(lambda x : int(x[0:4]))
trade_df['month'] = trade_df['기간'].apply(lambda x : int(x[5:7]))
trade_df.drop(columns = '기간', inplace=True)
trade_df

,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
1,양배추,184650,94,395802,90,4,2019,1
2,당근,23150,22,7466150,2955,-2934,2019,1
3,감귤,58368,172,0,0,172,2019,1
4,꽃양배추와 브로콜리(broccoli),780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
201,꽃양배추와 브로콜리(broccoli),24,0,332640,352,-352,2023,2
202,양배추,13188,13,377456,104,-91,2023,2
203,당근,22510,20,9260020,3758,-3737,2023,2
204,순무,4000,4,2,0,4,2023,2


In [33]:
trade_df['품목명'].unique()

array(['꽃양배추와 브로콜리(broccoli)', '양배추', '당근', '감귤', '순무'], dtype=object)

In [34]:
train_df['item'].unique()

array(['TG', 'CR', 'CB', 'RD', 'BC'], dtype=object)

In [35]:
lst = ['BC', 'CB', 'CR', 'TG', 'RD']
for i, item in enumerate(trade_df['품목명'].unique()):
    trade_df.loc[trade_df['품목명'] == item, '품목명'] = lst[i]

trade_df

,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,BC,160,1,638913,563,-562,2019,1
1,CB,184650,94,395802,90,4,2019,1
2,CR,23150,22,7466150,2955,-2934,2019,1
3,TG,58368,172,0,0,172,2019,1
4,BC,780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
201,BC,24,0,332640,352,-352,2023,2
202,CB,13188,13,377456,104,-91,2023,2
203,CR,22510,20,9260020,3758,-3737,2023,2
204,RD,4000,4,2,0,4,2023,2


In [36]:
trade_df = trade_df.rename(columns={
    '품목명': 'item',
    '수출 중량': 'out_weight',
    '수출 금액': 'out_value',
    '수입 중량': 'in_weight',
    '수입 금액': 'in_value',
    '무역수지': 'tradebalance'
})

trade_df.head()

,item,out_weight,out_value,in_weight,in_value,tradebalance,year,month
0,BC,160,1,638913,563,-562,2019,1
1,CB,184650,94,395802,90,4,2019,1
2,CR,23150,22,7466150,2955,-2934,2019,1
3,TG,58368,172,0,0,172,2019,1
4,BC,780,1,396870,399,-398,2019,2


In [37]:
train_df = train_df.merge(trade_df, on=['year', 'month', 'item'], how='left')
test_df = test_df.merge(trade_df, on=['year', 'month', 'item'], how='left')

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,total,month_week_mean_total,month_week_std_total,month_mean_total,month_std_total,out_weight,out_value,in_weight,in_value,tradebalance
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,104718528.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,35200000.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,40440000.0,5.244109e+07,5.790161e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,...,211741920.0,1.706946e+08,5.079797e+07,1.476583e+08,9.325641e+07,4000.0,4.0,2.0,0.0,4.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,...,224071380.0,1.706946e+08,5.079797e+07,1.476583e+08,9.325641e+07,4000.0,4.0,2.0,0.0,4.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,...,219830520.0,1.526513e+08,8.587015e+07,1.469691e+08,7.611450e+07,NaN,NaN,NaN,NaN,NaN
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,...,249586060.0,1.526513e+08,8.587015e+07,1.469691e+08,7.611450e+07,NaN,NaN,NaN,NaN,NaN


In [38]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(0, 0)

In [39]:
# 품목별 평균 가격
item_mean_price = train_df.groupby('item')['price'].mean().reset_index()
item_mean_price.columns = ['item', 'item_mean_price']

train_df = train_df.merge(item_mean_price, on='item', how='left')
test_df = test_df.merge(item_mean_price, on='item', how='left')

# 법인별 평균 가격
corp_mean_price = train_df.groupby('corporation')['price'].mean().reset_index()
corp_mean_price.columns = ['corporation', 'corp_mean_price']

train_df = train_df.merge(corp_mean_price, on='corporation', how='left')
test_df = test_df.merge(corp_mean_price, on='corporation', how='left')

# 지역별 평균 가격
location_mean_price = train_df.groupby('location')['price'].mean().reset_index()
location_mean_price.columns = ['location', 'location_mean_price']

train_df = train_df.merge(location_mean_price, on='location', how='left')
test_df = test_df.merge(location_mean_price, on='location', how='left')

In [40]:
from sklearn.preprocessing import LabelEncoder

cols = ['item', 'corporation', 'location']

for col in cols:
    enc = LabelEncoder()
    train_df[col]=enc.fit_transform(train_df[col])
    test_df[col]=enc.transform(test_df[col])

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_mean_total,month_std_total,out_weight,out_value,in_weight,in_value,tradebalance,item_mean_price,corp_mean_price,location_mean_price
0,TG_A_J_20190101,2019-01-01,4,0,0,0.0,0.0,2019,1,1,...,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148
1,TG_A_J_20190102,2019-01-02,4,0,0,0.0,0.0,2019,1,1,...,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148
2,TG_A_J_20190103,2019-01-03,4,0,0,60601.0,1728.0,2019,1,1,...,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148
3,TG_A_J_20190104,2019-01-04,4,0,0,25000.0,1408.0,2019,1,1,...,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148
4,TG_A_J_20190105,2019-01-05,4,0,0,32352.0,1250.0,2019,1,1,...,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148


In [41]:
train_df.columns

Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'week', 'day', 'day_of_week', 'day_of_year',
       'sin_dayofweek', 'cos_dayofweek', 'month_week_mean', 'month_week_max',
       'month_week_std', 'month_mean', 'month_max', 'month_std',
       'last_month_mean', 'month_mean_supply', 'month_std_supply',
       'month_week_mean_supply', 'month_week_std_supply', 'total',
       'month_week_mean_total', 'month_week_std_total', 'month_mean_total',
       'month_std_total', 'out_weight', 'out_value', 'in_weight', 'in_value',
       'tradebalance', 'item_mean_price', 'corp_mean_price',
       'location_mean_price'],
      dtype='object')

In [42]:
test_df.head()

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,...,month_mean_total,month_std_total,out_weight,out_value,in_weight,in_value,tradebalance,item_mean_price,corp_mean_price,location_mean_price
0,TG_A_J_20230304,2023-03-04,4,0,0,2023,3,9,4,5,...,10861753.7,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148
1,TG_A_J_20230305,2023-03-05,4,0,0,2023,3,9,5,6,...,10861753.7,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148
2,TG_A_J_20230306,2023-03-06,4,0,0,2023,3,10,6,0,...,10861753.7,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148
3,TG_A_J_20230307,2023-03-07,4,0,0,2023,3,10,7,1,...,10861753.7,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148
4,TG_A_J_20230308,2023-03-08,4,0,0,2023,3,10,8,2,...,10861753.7,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148


In [43]:
target = train_df['price']
target

0           0.0
1           0.0
2        1728.0
3        1408.0
4        1250.0
          ...  
59392     468.0
59393     531.0
59394     574.0
59395     523.0
59396     529.0
Name: price, Length: 59397, dtype: float64

In [44]:
train_df['item_id'] = train_df.ID.str[0:6]
train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_std_total,out_weight,out_value,in_weight,in_value,tradebalance,item_mean_price,corp_mean_price,location_mean_price,item_id
0,TG_A_J_20190101,2019-01-01,4,0,0,0.0,0.0,2019,1,1,...,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148,TG_A_J
1,TG_A_J_20190102,2019-01-02,4,0,0,0.0,0.0,2019,1,1,...,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148,TG_A_J
2,TG_A_J_20190103,2019-01-03,4,0,0,60601.0,1728.0,2019,1,1,...,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148,TG_A_J
3,TG_A_J_20190104,2019-01-04,4,0,0,25000.0,1408.0,2019,1,1,...,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148,TG_A_J
4,TG_A_J_20190105,2019-01-05,4,0,0,32352.0,1250.0,2019,1,1,...,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,3177.442022,1141.704458,1062.35148,TG_A_J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,3,5,0,452440.0,468.0,2023,2,9,...,9.325641e+07,4000.0,4.0,2.0,0.0,4.0,187.073539,259.121799,1062.35148,RD_F_J
59393,RD_F_J_20230228,2023-02-28,3,5,0,421980.0,531.0,2023,2,9,...,9.325641e+07,4000.0,4.0,2.0,0.0,4.0,187.073539,259.121799,1062.35148,RD_F_J
59394,RD_F_J_20230301,2023-03-01,3,5,0,382980.0,574.0,2023,3,9,...,7.611450e+07,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J
59395,RD_F_J_20230302,2023-03-02,3,5,0,477220.0,523.0,2023,3,9,...,7.611450e+07,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J


In [45]:
test_df['item_id'] = test_df.ID.str[0:6]
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,...,month_std_total,out_weight,out_value,in_weight,in_value,tradebalance,item_mean_price,corp_mean_price,location_mean_price,item_id
0,TG_A_J_20230304,2023-03-04,4,0,0,2023,3,9,4,5,...,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
1,TG_A_J_20230305,2023-03-05,4,0,0,2023,3,9,5,6,...,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
2,TG_A_J_20230306,2023-03-06,4,0,0,2023,3,10,6,0,...,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
3,TG_A_J_20230307,2023-03-07,4,0,0,2023,3,10,7,1,...,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
4,TG_A_J_20230308,2023-03-08,4,0,0,2023,3,10,8,2,...,1.059350e+07,0.0,0.0,0.0,0.0,0.0,3177.442022,1141.704458,1062.35148,TG_A_J
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,3,5,0,2023,3,13,27,0,...,7.611450e+07,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J
1088,RD_F_J_20230328,2023-03-28,3,5,0,2023,3,13,28,1,...,7.611450e+07,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J
1089,RD_F_J_20230329,2023-03-29,3,5,0,2023,3,13,29,2,...,7.611450e+07,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J
1090,RD_F_J_20230330,2023-03-30,3,5,0,2023,3,13,30,3,...,7.611450e+07,0.0,0.0,0.0,0.0,0.0,187.073539,259.121799,1062.35148,RD_F_J


In [46]:
train_dropcols = ['ID', 'supply']
test_dropcols = ['ID']

In [47]:
train_ft = train_df.drop(columns = train_dropcols)
test_ft = test_df.drop(columns = test_dropcols)

In [48]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

data = TimeSeriesDataFrame(train_ft)
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price",
    eval_metric="RMSE"
)
# seed 고정
predictor.fit(data, random_seed=SEED, presets="best_quality", time_limit=3600*3.5, hyperparameter_tune_kwargs={'num_trials': 20,
                                'scheduler': 'local',
                                'searcher': 'auto'})

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: best_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': {'num_trials': 20,
                                'scheduler': 'local',
                                'searcher': 'auto'},
 'hyperparameters': 'best_quality',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price',
 'time_limit': 12600.0,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231117_024418/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be 

  0%|          | 0/20 [00:00<?, ?it/s]

	Trained 11 models while tuning DeepAR.
	-737.2365     = Validation score (-RMSE)
	1226.31 s     = Total tuning time
Hyperparameter tuning model: TemporalFusionTransformer. Tuning model for up to 1259.99s of the 12599.87s remaining.
	-710.3106     = Validation score (-RMSE)
	370.61  s     = Training runtime
	0.14    s     = Validation (prediction) runtime
Hyperparameter tuning model: PatchTST. Tuning model for up to 1259.99s of the 12599.87s remaining.
	-1054.5956    = Validation score (-RMSE)
	112.94  s     = Training runtime
	0.25    s     = Validation (prediction) runtime
Hyperparameter tuning model: DirectTabular. Tuning model for up to 1259.99s of the 12599.87s remaining.
	-977.5140     = Validation score (-RMSE)
	71.97   s     = Training runtime
	0.73    s     = Validation (prediction) runtime
Hyperparameter tuning model: AutoARIMA. Tuning model for up to 1259.99s of the 12599.87s remaining.
	-845.6888     = Validation score (-RMSE)
	0.11    s     = Training runtime
	301.90  s   

In [49]:
predictor.get_model_names()

['Naive',
 'SeasonalNaive',
 'Theta',
 'AutoETS',
 'RecursiveTabular',
 'DeepAR/T1',
 'DeepAR/T2',
 'DeepAR/T3',
 'DeepAR/T4',
 'DeepAR/T5',
 'DeepAR/T6',
 'DeepAR/T7',
 'DeepAR/T8',
 'DeepAR/T9',
 'DeepAR/T10',
 'DeepAR/T11',
 'TemporalFusionTransformer',
 'PatchTST',
 'DirectTabular',
 'AutoARIMA',
 'WeightedEnsemble']

In [50]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	59.30   s     = Training runtime
Fitting model: DeepAR/T1_FULL | Skipping fit via cloning parent ...
Fitting model: DeepAR/T2_FULL | Skipping fit via cloning parent ...
Fitting model: DeepAR/T3_FULL | Skipping fit via cloning parent ...
Fitting model: DeepAR/T4_FULL | Skipping fit via cloning parent ...
Fitting model: DeepAR/T5_FULL | Skipping fit via cloning parent ...
Fitting model: DeepAR/T6_FULL | Skipping fit v

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR/T1': 'DeepAR/T1_FULL',
 'DeepAR/T2': 'DeepAR/T2_FULL',
 'DeepAR/T3': 'DeepAR/T3_FULL',
 'DeepAR/T4': 'DeepAR/T4_FULL',
 'DeepAR/T5': 'DeepAR/T5_FULL',
 'DeepAR/T6': 'DeepAR/T6_FULL',
 'DeepAR/T7': 'DeepAR/T7_FULL',
 'DeepAR/T8': 'DeepAR/T8_FULL',
 'DeepAR/T9': 'DeepAR/T9_FULL',
 'DeepAR/T10': 'DeepAR/T10_FULL',
 'DeepAR/T11': 'DeepAR/T11_FULL',
 'TemporalFusionTransformer': 'TemporalFusionTransformer_FULL',
 'PatchTST': 'PatchTST_FULL',
 'DirectTabular': 'DirectTabular_FULL',
 'AutoARIMA': 'AutoARIMA_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [51]:
predictor.get_model_names()

['Naive',
 'SeasonalNaive',
 'Theta',
 'AutoETS',
 'RecursiveTabular',
 'DeepAR/T1',
 'DeepAR/T2',
 'DeepAR/T3',
 'DeepAR/T4',
 'DeepAR/T5',
 'DeepAR/T6',
 'DeepAR/T7',
 'DeepAR/T8',
 'DeepAR/T9',
 'DeepAR/T10',
 'DeepAR/T11',
 'TemporalFusionTransformer',
 'PatchTST',
 'DirectTabular',
 'AutoARIMA',
 'WeightedEnsemble',
 'Naive_FULL',
 'SeasonalNaive_FULL',
 'Theta_FULL',
 'AutoETS_FULL',
 'RecursiveTabular_FULL',
 'DeepAR/T1_FULL',
 'DeepAR/T2_FULL',
 'DeepAR/T3_FULL',
 'DeepAR/T4_FULL',
 'DeepAR/T5_FULL',
 'DeepAR/T6_FULL',
 'DeepAR/T7_FULL',
 'DeepAR/T8_FULL',
 'DeepAR/T9_FULL',
 'DeepAR/T10_FULL',
 'DeepAR/T11_FULL',
 'TemporalFusionTransformer_FULL',
 'PatchTST_FULL',
 'DirectTabular_FULL',
 'AutoARIMA_FULL',
 'WeightedEnsemble_FULL']

In [52]:
pred = predictor.predict(data, random_seed=SEED, model='WeightedEnsemble_FULL')
pred

INFO:lightning_fabric.utilities.seed:Global seed set to 42


mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3228.216187  1118.923523  2133.344920  2620.728497   
        2023-03-05   435.316721  -795.517136  -343.704608   -29.054971   
        2023-03-06  3329.228961   621.697979  1811.647568  2415.903220   
        2023-03-07  3370.918869   524.308915  1795.159246  2449.937387   
        2023-03-08  3318.981490   245.577033  1597.075854  2297.225028   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   555.181486    82.567089   277.364892   393.438065   
        2023-03-28   563.097497    69.981335   277.975061   395.460842   
        2023-03-29   559.067374    53.983134   269.633811   387.987995   
        2023-03-30   552.267787    38.196009   254.698666   377.579843   
        2023-03-31   557.881660    33.795881   249.774790   375.258649   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2939.262081  3211.571838  3469.932744  3762.496906   
        2023-03-05   194.282721   435.232428   670.458747   927.948682   
        2023-03-06  2837.013569  3227.843463  3620.843766  4072.440688   
        2023-03-07  2932.049232  3367.210129  3807.734308  4318.029320   
        2023-03-08  2830.194859  3326.485091  3776.813165  4326.199195   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   474.721282   553.274115   639.171506   727.563274   
        2023-03-28   483.491017   564.988152   649.004430   742.926306   
        2023-03-29   477.529275   562.602706   650.307155   745.341219   
        2023-03-30   469.897419   556.472888   645.598796   741.924138   
        2023-03-31   470.694003   561.681351   651.723142   747.825862   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4102.933583  4602.581604  
        2023-03-05  1227.088441  1668.236087  
        2023-03-06  4543.183095  5220.884485  
        2023-03-07  4842.280909  5642.841904  
        2023-03-08  4899.870707  5796.949150  
...                         ...          ...  
RD_F_J  2023-03-27   831.972789   994.727966  
        2023-03-28   850.216805  1014.737065  
        2023-03-29   854.204397  1020.910685  
        2023-03-30   851.243868  1021.425751  
        2023-03-31   863.496809  1032.050913  

[1092 rows x 10 columns]

In [ ]:
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission

,ID,answer
0,TG_A_J_20230304,3242.523713
1,TG_A_J_20230305,389.450981
2,TG_A_J_20230306,3255.184581
3,TG_A_J_20230307,3395.988407
4,TG_A_J_20230308,3314.301545
...,...,...
1087,RD_F_J_20230327,513.469446
1088,RD_F_J_20230328,515.985473
1089,RD_F_J_20230329,516.326125
1090,RD_F_J_20230330,509.636177


In [ ]:
submission.to_csv(f"{DATA_PATH}submission_autogluon_ver2.csv", index=False)